In [19]:
daily_returns = sc.textFile("s3n://msan-usf-gproject/daily_returns.csv")
header = daily_returns.first()

rdd = daily_returns.filter(lambda x: x != header)

In [20]:
def toDouble(x):
    try:
        return float(x)
    except ValueError:
        return -9999.0

def clipper(val):
    if val < -0.075:
        return -0.075
    elif val > 0.075:
        return 0.075
    else:
        return val
    

rddSplit = rdd.map(lambda x: x.split(','))
rddDoubles = rddSplit.map(lambda x: tuple([toDouble(val) for val in x]))
df = sqlContext.createDataFrame(rddDoubles,  header.split(','))

In [23]:
import pandas as pd
global_medians = pd.read_csv('feature_medians.csv')

,feature,median
0,id,1.090000e+03
1,timestamp,4.780000e+02
2,derived_0,2.433180e-03
3,derived_1,7.520909e-03
4,derived_2,2.601965e-02
5,derived_3,4.835723e-03
6,derived_4,-1.984233e-02
7,fundamental_0,-2.846959e-02
8,fundamental_1,-1.414638e-02
9,fundamental_2,-6.989566e-02


In [24]:
from pyspark.sql.functions import * 

for col in df.columns[2:-1]:
    median = float(global_medians.loc[global_medians.feature==col, 'median'])
    df = df.replace(subset=col, to_replace=0.0, value=median)
    
    
df_train = df.where('timestamp < 906')
df_test = df.where('timestamp >= 906')

In [27]:
################# RandomForestation ################################

##### Assemble features ###########
from pyspark.ml.feature import VectorAssembler
va_train = VectorAssembler(outputCol="features", inputCols=df_train.columns[2:-1])
va_test = VectorAssembler(outputCol="features", inputCols=df_test.columns[2:-1])
train = va_train.transform(df_train).select("features", "y").cache()
test = va_test.transform(df_test).select("features", "y").cache()

train = train.withColumnRenamed(existing='y', new='label')
test = test.withColumnRenamed(existing='y', new='label')

##### train randomforestation ###########
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(numTrees=200, maxDepth=5)
rfmodel = rf.fit(train)

##### predict
rfpredcitions = rfmodel.transform(test)
predictions = rfpredcitions.select("prediction").rdd.map(lambda r:  clipper(r[0]) ).collect()

In [28]:
#### get actual returns
labels = rfpredcitions.select("label").rdd.map(lambda r: r[0]).collect()

In [37]:
################### Linear Reregression ################################

df_reg_train = df_train.select('id', 'timestamp', 'technical_20', 'technical_30', 'y')
df_reg_test = df_test.select('id', 'timestamp', 'technical_20', 'technical_30', 'y')


##### Assemble features #############
from pyspark.ml.feature import VectorAssembler
va_train = VectorAssembler(outputCol="features", inputCols=df_reg_train.columns[2:-1])
va_test = VectorAssembler(outputCol="features", inputCols=df_reg_test.columns[2:-1])
train = va_train.transform(df_reg_train).select("features", "y").cache()
test = va_test.transform(df_reg_test).select("features", "y").cache()

train = train.withColumnRenamed(existing='y', new='label')
test = test.withColumnRenamed(existing='y', new='label')

##### train linear regression model
from pyspark.ml.regression import LinearRegression
reg = LinearRegression()
reg_estimate = reg.fit(train)

##### predict
reg_predictions = reg_estimate.transform(test)
reg_pred = reg_predictions.select('prediction').rdd.map(lambda r: clipper(r[0])).collect()

In [47]:
import numpy as np


predictions_f = ( np.array(predictions) * .85 + np.array(reg_pred) *0.15)
labels = np.array(labels)

def rvalue(pred, y):
    mu = np.mean(y)
    
    rsquare = 1 - np.sum( ( y - pred )**2 ) / np.sum( (y - mu )**2)
    r = np.sign(rsquare) * np.sqrt( np.abs(rsquare) )
    
    return r

print "R-value:",rvalue(predictions_f, labels)

R-value: 0.0268568840424
